# Reverse Logistics MVP

As a first step for my Reverse Logistics project, I will create a MVP which will show the trade lanes where this model can be applied. It will only compare the Tender itself for one customer. 

In order to know if Reverse Logistics can be applied for this customer and its lanes, it has to fulfill the following:
     * Origin City = Destination City (i.eg Lane 1 has an origin A and Lane 2 has a destination A)
     * By Definition, Origin and Destination country has to be the same
     * Annual volume has to be higher than 54, ~1 containers per week in the matching lanes.

This first MVP will exclude close cities (Google Maps API) and multiple combinations

## 1. Data exploration

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
file_root = 'Reverse_Logistics_Template.xlsx'

In [4]:
tender = pd.read_excel(file_root)

In [5]:
tender.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_City,Destination_City,Origin_Adress,Destination_Adress,Equipment_Type,Unnamed: 9,Shipping_Terms,Commodity,Volume
0,CustomerA,Item-0001,FR,US,Le Havre,"New York, NY",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,3080.0
1,CustomerA,Item-0002,FR,SG,Le Havre,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,1250.0
2,CustomerA,Item-0003,FR,US,Le Havre,"Oakland, CA",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,770.0
3,CustomerA,Item-0004,GB,KR,North West England,Busan,NaN,NaN,40' Standard Dry,NaN,Door-Quay excl. DTHC,Beer,665.0
4,CustomerA,Item-0005,FR,SG,Fos-sur-Mer,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,610.0


In [6]:
#This will show as the Exporter Countries, United Kingdom, France and Irelande as Top3
top_exporters = tender.groupby('Origin_Country').sum()
top_exporters.sort_values('Volume', ascending=False)

,Unnamed: 9,Volume
Origin_Country,,
GB,0.0,16049.8
FR,0.0,9242.0
IE,0.0,3231.2
NL,0.0,2897.5
IT,0.0,2032.0
US,0.0,420.0
CL,0.0,308.0
TH,0.0,228.0
NZ,0.0,221.0


In [7]:
#This will show as the Importers Countries, United States, Singapore and China as Top3
top_importers = tender.groupby('Destination_Country').sum()
top_importers.sort_values('Volume', ascending=False)

,Unnamed: 9,Volume
Destination_Country,,
US,0.0,15786.1
SG,0.0,4182.4
CN,0.0,1813.4
KR,0.0,1553.0
AU,0.0,1286.8
ZA,0.0,1008.3
CO,0.0,977.0
MX,0.0,959.2
BR,0.0,843.0


## 2. Cleaning Lanes with <54 volume

In [8]:
tender_cleaned = tender[~(tender['Volume'] <54)]
tender_cleaned.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_City,Destination_City,Origin_Adress,Destination_Adress,Equipment_Type,Unnamed: 9,Shipping_Terms,Commodity,Volume
0,CustomerA,Item-0001,FR,US,Le Havre,"New York, NY",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,3080.0
1,CustomerA,Item-0002,FR,SG,Le Havre,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,1250.0
2,CustomerA,Item-0003,FR,US,Le Havre,"Oakland, CA",NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,770.0
3,CustomerA,Item-0004,GB,KR,North West England,Busan,NaN,NaN,40' Standard Dry,NaN,Door-Quay excl. DTHC,Beer,665.0
4,CustomerA,Item-0005,FR,SG,Fos-sur-Mer,Singapore,NaN,NaN,40' Standard Dry,NaN,Quay-Quay incl. DTHC,French Wines & Spirits,610.0


In [9]:
rows_removed = tender['Lane_ID'].count() - tender_cleaned['Lane_ID'].count()
rows_removed

1037

In [10]:
tender_cleaned['Lane_ID'].count()
#There are 121 valid lanes for Customer A to apply Reverse Logistics to its own Tender.

121

## 3. Matching the Trade Lanes

In [11]:
#Let's check if any city has more than 54 loads per year both as importer and exporter

In [42]:
city_exports = tender.groupby('Origin_City').sum()
city_exports.sort_values('Volume', ascending=False)

,Unnamed: 9,Volume
Origin_City,,
Le Havre,0.0,8411.0
"Grangemouth, FAL",0.0,5316.4
"Greenock, STD",0.0,4726.2
Dublin,0.0,3231.2
Schiedam,0.0,2815.2
North West England,0.0,2468.0
Santa Vittoria D'Alba,0.0,2030.0
"Belfast, BFS",0.0,1929.4
"Coatbridge, STD",0.0,1437.6


In [43]:
city_exports = city_exports.rename({'Volume': 'Export_Volume'}, axis=1)

In [44]:
del city_exports['Unnamed: 9']

In [45]:
#Let's do the same for imports
city_imports = tender.groupby('Destination_City').sum()
city_imports.sort_values('Volume', ascending=False)
city_imports = city_imports.rename({'Volume': 'Import_Volume'}, axis=1)
del city_imports['Unnamed: 9']

In [87]:
#Let's Concatenate both tables:
total_volume = pd.concat([city_exports, city_imports], axis=1, join="inner")
total_volume.reset_index(level=0, inplace=True)
total_volume = total_volume.rename({'index': 'City'}, axis=1)
total_volume.head()

,City,Export_Volume,Import_Volume
0,Alexandria,50.0,98.0
1,Altamira,120.0,85.6
2,Bangkok,228.0,7.0
3,Buenos Aires,146.0,18.0
4,Busan,44.0,1553.0


In [90]:
#Let's see in which cities we can apply the reverse logistics (not taking into account the proximity of the cities, yet), for this they need to have at least 54 ctrs in both imports and exports
total_volume_cleaned = total_volume[~(total_volume['Export_Volume'] <54) & ~(total_volume['Import_Volume'] <54) ]

total_volume_cleaned.head()

,City,Export_Volume,Import_Volume
1,Altamira,120.0,85.6
5,Dublin,3231.2,54.0
7,Le Havre,8411.0,245.0


In [94]:
#Let's see all the lanes with Altarmia as Origin City by equipment type
Altamira_Export = tender[tender['Origin_City']=='Altamira']
Altamira_Export.groupby('Equipment_Type')['Volume'].sum()

Equipment_Type
20' Standard Dry    76.0
40' Standard Dry    44.0
Name: Volume, dtype: float64

In [95]:
#Let's do the same by Destination
Altamira_Import = tender[tender['Destination_City']=='Altamira']
Altamira_Import.groupby('Equipment_Type')['Volume'].sum()

Equipment_Type
20' Standard Dry    37.5
40' Standard Dry    48.1
Name: Volume, dtype: float64

## Conclusion
In this case, we wouldn't be able to do reverse logistics for Altarmia since we do not have enough volume in order to make it profitable. Also, it looks like for this customer, it is going to be complicated to apply it.